Connect to database

In [1]:
import datajoint as dj

dj.config["database.host"] = "gl-ash.biostr.washington.edu"
dj.config["database.user"] = "gabby"
dj.config["database.port"] = 3306

dj.conn()

/home/gl-willow/mambaforge/envs/gabby_spyglass_env2/lib/python3.10/site-packages/datajoint/plugin.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2026-02-18 16:33:13,384][INFO]: DataJoint 0.14.6 connected to gabby@gl-ash.biostr.washington.edu:3306


DataJoint connection (connected) gabby@gl-ash.biostr.washington.edu:3306

Imports

In [2]:
import os
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import ghostipy as gsp

import spyglass.common as sgc
import spyglass.lfp as lfp

import figpack.views as vv

from gl_spyglass.utils.common_neural_functions import validate_references, apply_referencing

In [9]:
os.environ['FIGPACK_BUCKET'] = 'gillespielab'

# TODO: update this with your own custom FIGPACK_API_KEY (contact Jeremy Magland if you don't have one)
os.environ['FIGPACK_API_KEY'] = '17979e3bc8880ea91520c1dc2e8ab6a97fc665893fe2728e9d642923c3dcf31c'

Select subj, date, epoch to plot

In [7]:
subj = 'pippin'
date = 20210421
epoch = 1

Set plotting parameters

In [16]:
ref_on = False
save_dir = '/home/gl-willow/Documents/gabby/gabby_analysis_gillespie/results/old_young_results/sleep_spectrograms/'

Calculate average CA1 spectrogram

Note: can use this code to calculate the spectrogram for any group of electrodes, just select the ones you want (i.e. only the strongest SWR CA1 electrodes, only the cortical electrodes, etc.)

In [17]:
# set corresponding nwb_file_name and interval_list_name
nwb_file_name = f'{subj}{date}_.nwb'
interval_list_name = (
    sgc.TaskEpoch() & {"nwb_file_name": nwb_file_name, "epoch": epoch}
).fetch1("interval_list_name")
pos_interval_list_name = (
    sgc.IntervalList()
    & {"nwb_file_name": nwb_file_name, "pipeline": "position"}
).fetch("interval_list_name")[epoch - 1]

# load in one electrode per electrode group for ca1 region, excluding bad channels
electrodes_df, val_can_refs = validate_references(nwb_file_name, is_copy=True)
electrodes_df = electrodes_df[electrodes_df['bad_channel'] == 'False']
electrodes_df = pd.DataFrame(
    [
        electrodes_df[electrodes_df['electrode_group_name'] == i].iloc[0]
        for i in np.unique(electrodes_df['electrode_group_name'].values)
    ]
)

# load in lfp
good_elecs_df = electrodes_df[
    (electrodes_df['bad_channel'] == 'False')
]
good_single_elecs_df = pd.DataFrame(
    [
        good_elecs_df[good_elecs_df["electrode_group_name"] == i].iloc[0]
        for i in np.unique(good_elecs_df["electrode_group_name"].values)
    ]
)
good_single_elecs = good_single_elecs_df['electrode_id'].values
lfp_electrode_group_name = 'good_single_elecs'

lfp_sampling_rate = 1_000

lfp_filter_name = 'LFP 0-400 Hz'
lfp_s_key = {
    'nwb_file_name': nwb_file_name,
    'lfp_electrode_group_name': lfp_electrode_group_name,
    'target_interval_list_name': interval_list_name,
    'filter_name': lfp_filter_name,
    'filter_sampling_rate': 30_000,  # sampling rate of the data (Hz)
    'target_sampling_rate': lfp_sampling_rate,  # sampling rate of the lfp output (Hz)
}

lfp_merge_id = (lfp.LFPOutput.LFPV1() & lfp_s_key).fetch1('merge_id')
lfp_key = {
    'merge_id': lfp_merge_id,
}
lfp_df = (lfp.LFPOutput & lfp_key).fetch1_dataframe()
lfp_df.columns = good_single_elecs  # rename lfp columns to their original elec ids

# apply referencing if referencing is on
if ref_on:
    lfp_df = apply_referencing(lfp_df, electrodes_df)

# get the start time of interval list to align with lfp time
int_start_time, int_end_time = (sgc.IntervalList() & {'nwb_file_name': nwb_file_name, 'interval_list_name': interval_list_name}).fetch1('valid_times')[0]
start_time_sec = lfp_df.index[0] - int_start_time

In [ ]:
# GENERATE CA1 SPECTROGRAM
# select all ca1 electrodes
all_ca1_elecs = electrodes_df.loc[electrodes_df['region_name'] == 'ca1', 'electrode_id'].values

# calculate for just one electrode first to get the right frequency and time bins
elec = all_ca1_elecs[0]
elec_df = lfp_df[elec]

data = elec_df.values

fs = lfp_sampling_rate

timestamps = elec_df.index.values - elec_df.index.values[0]

coefs_cwt, _, f_cwt, t_cwt, _ = gsp.cwt(data=data,
                                freq_limits=[0.5, 200],
                                fs=fs,
                                timestamps=timestamps)

psd_cwt = coefs_cwt.real**2 + coefs_cwt.imag**2
psd_cwt /= np.max(psd_cwt)

spectral_sampling_rate = 1 / np.mean(np.diff(t_cwt))

# load in avg ca1 spectrogram if it already exists, otherwise calculate and save
if os.path.exists(os.path.join(save_dir, f'{nwb_file_name}_{interval_list_name}_avg_ca1_psd.npy')):
    print('already exists!')
    avg_ca1_psd = np.load(os.path.join(save_dir, f'{nwb_file_name}_{interval_list_name}_avg_ca1_psd.npy'))
else:
    print('calculating avg ca1 psd...')
    all_psds = np.zeros((len(all_ca1_elecs), psd_cwt.shape[0], psd_cwt.shape[1]))

    for e, elec in tqdm(enumerate(all_ca1_elecs)):
        data = lfp_df[elec].values
        coefs_cwt, _, f_cwt, t_cwt, _ = gsp.cwt(data=data,
                                    freq_limits=[0.5, 200],
                                    fs=fs,
                                    timestamps=timestamps)

        psd_cwt = coefs_cwt.real**2 + coefs_cwt.imag**2
        psd_cwt /= np.max(psd_cwt)
        all_psds[e] = psd_cwt

    avg_ca1_psd = all_psds.mean(axis=0)
    np.save(os.path.join(save_dir, f'{nwb_file_name}_{interval_list_name}_avg_ca1_psd.npy'),
            avg_ca1_psd)

already exists!


Plot and show spectrogram

In [21]:
avg_ca1_spectrogram = vv.Spectrogram(
    start_time_sec=start_time_sec,
    sampling_frequency_hz=spectral_sampling_rate,
    frequencies=f_cwt[::-1],
    data=avg_ca1_psd[::-1, :].T,
)
avg_ca1_spectrogram.show(title=f"{nwb_file_name} {interval_list_name} {'referenced' if ref_on else 'unreferenced'} avg CA1 spectrogram", upload=True)

Stored Spectrogram with 5 downsampled levels:
  Original: (1811725, 87) (chunks: (8192, 87))
  Factor 4: (452932, 87) (chunks: (8192, 87))
  Factor 16: (113233, 87) (chunks: (8192, 87))
  Factor 64: (28309, 87) (chunks: (8192, 87))
  Factor 256: (7078, 87) (chunks: (7078, 87))
  Factor 1024: (1770, 87) (chunks: (1770, 87))
Found 13 files to upload, total size: 617.51 MB
Uploading 13 files in batches of 20 with up to 16 concurrent uploads per batch...
Processing batch 1/1 (13 files)...
Uploaded 1/13: assets/index-GPjx4QpG.css
Uploaded 2/13: extension_manifest.json
Uploaded 3/13: assets/neurosift-logo-CLsuwLMO.png
Uploaded 4/13: data.zarr/.zmetadata
Uploaded 5/13: index.html
Uploaded 6/13: assets/index-BY1Hwjm4.js
Uploaded 7/13: data.zarr/_consolidated_6.dat
Uploaded 8/13: data.zarr/_consolidated_0.dat
Uploaded 9/13: data.zarr/_consolidated_2.dat
Uploaded 10/13: data.zarr/_consolidated_5.dat
Uploaded 11/13: data.zarr/_consolidated_1.dat
Uploaded 12/13: data.zarr/_consolidated_4.dat
Uploa

'https://gillespielab.figpack.org/figures/default/0f174fb7e425f42cee1da275/index.html'

Combine with any other views you want!

Keep in mind you can also play with the frequency bins and frequency limits within the ghostipy continuous wavelet transform function (`gsp.cwt`)